# TP4 : KPPV

Le parcours du combattant se termine bientôt, je vous le promets. Voici le dernier type de classifieur qu'on verra ensemble : KPPV, pour "K plus proches voisins.

In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas
import progressbar

from utils import Model, split_data, read_data

Le principe est super simple ! Vraiment ! 

Pour chaque échantillon à prédire, on va calculer la distance de celui-ci à chaque échantillon du jeu d'apprentissage. Ensuite, on sélectionnera les K plus proches de cet échantillon. Pour finir, on définit la classe de cet échantillon à prédire en fonction des classes des K échantillons les plus proches.

**La question préliminaire :** Alors, KPPV ? Méthode paramétrique ou non paramétrique ?

**_Réponse :_** C'est une méthode non paramétrique ! Ici, on ne fait AUCUNE supposition sur la distribution des données, c'est avec les échantillons qu'on en fait une estimation. Même conclusion que pour Parzen ;)

## Partie 1 : La démocratie, c'est la clé

Après avoir sélectionné les K points les plus proches, il faut faire un vote pour attribuer une classe à l'échantillon à prédire. Pour cela, on va utiliser 2 méthodes de vote.

**1)** Première méthode de vote : le vote à la majorité... On choisit la classe majoritaire... Voilà quoi...

Ecrivez une fonction qui prendra en entrée une liste des classes de K points (les plus proches), ainsi que la liste des classes possibles, et qui retournera la liste de classes triée par rapport au nombre d'occurences, du plus grand au plus petit.

In [2]:
# A compléter
# Fonction de vote majoritaire
def vote_majoritaire(k_classes,classes):
    counts = np.zeros((len(classes)))
    for i,c in enumerate(classes):
        counts[i] = k_classes.count(c)
    ind = np.argsort(counts)[::-1]
    return np.array(classes)[ind].tolist()

Testez votre fonction maintenant avec ces quelques tests unitaires réalisés par mes soins :D

In [3]:
# Test 1
classes = [1,2,3,4,5]
A = [1,1,2,2,2,2,3,4,4,4,5,5,5,5,5]
assert vote_majoritaire(A,classes)==[5,2,4,1,3], "Test 1 non fonctionnel"

# Test 2
A = [1,1,2,4,4,4,5,5,5,5]
assert vote_majoritaire(A,classes)==[5,4,1,2,3], "Test 2 non fonctionnel"

**2)** Deuxième méthode de vote : le vote à l'unanimité... Si les K échantillons les plus proches ont la même classe, on retourne cette classe, sinon, on retourne None. Ecrivez cette fonction.

In [4]:
# A compléter
# Fonction de vote unanime
def vote_unanime(k_classes,classes):
    if len(np.unique(k_classes))==1:
        return k_classes[0]
    else:
        return None

Hop hop hop ! On teste la fonction avant la prochaine étape !

In [5]:
# Test 1
A = [4,4,4,4]
assert vote_unanime(A,classes)==4, "Test 1 non fonctionnel"

# Test 2
A = [1,1,1,1,2]
assert vote_unanime(A,classes) is None, "Test 2 non fonctionnel"

## Partie 2 : Construction du modèle KPPV

**1)** Créez le classifieur KPPV :D

Lors de la prédiction d'un échantillon, on calculera la distance de cet échantillon avec les échantillons d'apprentissage et on sélectionnera les k plus proches puis on votera pour définir sa classe. On utilisera la distance euclidienne.

Le constructeur prendra donc 2 paramètres : K (le nombre d'échantillons à sélectionner pour le vote), et le mode de vote à utiliser (majoritaire ou unanime)

*Note :* Il va falloir redéfinir la fonction de métriques pour le vote à l'unanimité pour 2 raisons : 

- Le vote renvoit une seule valeur, et non une liste de classes triées...
- Il se peut que le vote renvoit une valeur None est donc que l'échantillon soit non classé

In [6]:
# A compléter
# Création de la classe de classifieur KPPV
class KPPV(Model):
    # Définition du constructeur
    def __init__(self,classes,vote='majoritaire',k=1):
        super(KPPV, self).__init__(classes)
        self.k = k
        assert vote in ['unanime','majoritaire']
        self.vote = vote
        if self.vote == 'unanime':
            self.vote_func = vote_unanime
        else:
            self.vote_func = vote_majoritaire
    
    # Définition de la fonction d'apprentissage
    def learning(self,app_x,app_y,app_labels):
        self.app_points = np.stack([app_x,app_y]).T
        self.app_labels = app_labels.to_numpy()
    
    def distance(self,A,B):
        return (A-B).T@(A-B)
    
    # Définition de la classe de prédiction
    def prediction(self,x,y):
        N = len(self.app_labels)
        point = np.array([x,y])
        labels = self.app_labels.copy()
        distances = np.array([self.distance(point,self.app_points[i,:]) for i in range(N)])
        k_classes_selec = []
        for i in range(self.k):
            ind_dist_min = np.argmin(distances)
            k_classes_selec.append(labels[ind_dist_min])
            distances = np.delete(distances,ind_dist_min)
            labels = np.delete(labels,ind_dist_min)
        return self.vote_func(k_classes_selec,self.classes)
    
    def metrics(self,gt,pred):
        if self.vote == 'majoritaire':
            return super(KPPV,self).metrics(gt,pred)
        else:
            CM = np.zeros((len(self.classes),len(self.classes)+1),dtype=np.uint16)
            top1 = 0
            top2 = 0
            for g,p in zip(gt,pred):
                if p is None:
                    CM[g-1,-1]+=1
                else:
                    if g == p:
                        top1+=1
                    CM[g-1,p-1]+=1
            return top1/len(gt), top1/len(gt), CM

**2)** Testez deux modèles KPPV (1 avec vote majoritaire, un avec vote unanime), sur le jeu de données 1, et avec comme paramètre k=2. Affichez les résultats (top1, top2 et matrice de confusion).

In [7]:
# A compléter
# Chargement des données tp1 app et dec
file_app = f'Archive/data_tp1_app.txt'
file_dec = f'Archive/data_tp1_dec.txt'
columns_labels = ['label','x','y']
data_app = read_data(file_app,columns_labels)
data_dec = read_data(file_dec,columns_labels)

classes = np.unique(data_app['label'])

# Création d'une instance de modèle classifieur KPPV avec vote majoritaire
kppv_majoritaire = KPPV(classes,'majoritaire',2)

# Création d'une instance de modèle classifieur KPPV avec vote unanime
kppv_unanime = KPPV(classes,'unanime',2)

# Apprentissage du classifieur avec les données d'apprentissage
app_x = data_app['x']
app_y = data_app['y']
app_labels = data_app['label']

kppv_majoritaire.learning(app_x,app_y,app_labels)
kppv_unanime.learning(app_x,app_y,app_labels)

# Evaluation du classifieur avec les données d'évaluation
dec_x = data_dec['x']
dec_y = data_dec['y']
dec_labels = data_dec['label']

print("TEST KPPV VOTE MAJORITAIRE")
top1,top2,CM = kppv_majoritaire.test(dec_x,dec_y,dec_labels)

print("TEST KPPV VOTE UNANIME")
top1,top2,CM = kppv_unanime.test(dec_x,dec_y,dec_labels)

TEST KPPV VOTE MAJORITAIRE
Top 1 :  0.992
Top 2 :  0.996
Matrice de confusion : 
    99   0   0   0   1
    0  98   0   0   2
    0   0 100   0   0
    0   0   0 100   0
    0   0   0   1  99
TEST KPPV VOTE UNANIME
Top 1 :  0.99
Top 2 :  0.99
Matrice de confusion : 
    99   0   0   0   1   0
    0  98   0   0   0   2
    0   0 100   0   0   0
    0   0   0 100   0   0
    0   0   0   0  98   2


**3)** Testez également sur les jeux de données 2 et 3 avec les 2 modes de vote. Complétez ensuite le tableau suivant. Pour simple comparaison, voici mes résultats obtenus :

| Jeu de données / Vote | top1 | top2 | nb points non classés |
|:---------------------:|:----:|:----:|:---------------------:|
| JDD 1 / Majoritaire  | 99.2% | 99.6% | 0 |
| JDD 1 / Unanime  | 99% | 99% | 4 |
| JDD 2 / Majoritaire  | 92.0% | 96.6% | 0 |
| JDD 2 / Unanime  | 88.4% | 88.4% | 41 |
| JDD 3 / Majoritaire  | 66.2% | 76.8% | 0 |
| JDD 3 / Unanime  | 50.6% | 50.6% | 174 |

In [8]:
# A compléter
# Chargement des données tp1 app et dec
for i in [2,3]:
    print(f"JDD {i}")
    file_app = f'Archive/data_tp{i}_app.txt'
    file_dec = f'Archive/data_tp{i}_dec.txt'
    columns_labels = ['label','x','y']
    data_app = read_data(file_app,columns_labels)
    data_dec = read_data(file_dec,columns_labels)

    classes = np.unique(data_app['label'])

    # Création d'une instance de modèle classifieur KPPV avec vote majoritaire
    kppv_majoritaire = KPPV(classes,'majoritaire',2)

    # Création d'une instance de modèle classifieur KPPV avec vote unanime
    kppv_unanime = KPPV(classes,'unanime',2)

    # Apprentissage du classifieur avec les données d'apprentissage
    app_x = data_app['x']
    app_y = data_app['y']
    app_labels = data_app['label']

    kppv_majoritaire.learning(app_x,app_y,app_labels)
    kppv_unanime.learning(app_x,app_y,app_labels)

    # Evaluation du classifieur avec les données d'évaluation
    dec_x = data_dec['x']
    dec_y = data_dec['y']
    dec_labels = data_dec['label']

    print("TEST KPPV VOTE MAJORITAIRE")
    top1,top2,CM = kppv_majoritaire.test(dec_x,dec_y,dec_labels)

    print("TEST KPPV VOTE UNANIME")
    top1,top2,CM = kppv_unanime.test(dec_x,dec_y,dec_labels)

JDD 2
TEST KPPV VOTE MAJORITAIRE
Top 1 :  0.92
Top 2 :  0.966
Matrice de confusion : 
   99  0  1  0  0
   0 87 13  0  0
   2  4 80  2 12
   0  0  3 95  2
   0  0  0  1 99
TEST KPPV VOTE UNANIME
Top 1 :  0.884
Top 2 :  0.884
Matrice de confusion : 
   99  0  0  0  0  1
   0 87  0  0  0 13
   2  4 79  1  6  8
   0  0  3 94  0  3
   0  0  0  1 83 16
JDD 3
TEST KPPV VOTE MAJORITAIRE
Top 1 :  0.662
Top 2 :  0.768
Matrice de confusion : 
   16 15 31 18 20
   6 69 13  1 11
   8  3 86  3  0
   7  1 17 70  5
   2  5  1  2 90
TEST KPPV VOTE UNANIME
Top 1 :  0.506
Top 2 :  0.506
Matrice de confusion : 
   16 11  5  7 11 50
   6 57  0  0  4 33
   8  1 58  1  0 32
   7  0  7 54  0 32
   2  3  0  0 68 27


| Jeu de données / Vote | top1 | top2 | nb points non classés |
|:---------------------:|:----:|:----:|:---------------------:|
| JDD 1 / Majoritaire  | 99.2% | 99.6% | 0 |
| JDD 1 / Unanime  | 99% | 99% | 4 |
| JDD 2 / Majoritaire  | 92.0% | 96.6% | 0 |
| JDD 2 / Unanime  | 88.4% | 88.4% | 41 |
| JDD 3 / Majoritaire  | 66.2% | 76.8% | 0 |
| JDD 3 / Unanime  | 50.6% | 50.6% | 174 |

## Partie 3 : Cross-validation

Alors, on choisit comment le k optimal ?

De la même manière que pour le paramètre h pour Parzen :D Avec une cross validation !

**1)** Finissez tranquillement ce TP en trouvant les paramètres optimaux de k pour chaque jeu de données via une cross validation à 3 dossiers, et pour chaque mode de vote (donc 6 k à trouver ;) ). Dès que c'est réalisé, remplissez le tableau. Comme d'habitude, je vous mets mes résultats (testés pour k entre 2 et 10).

*Note :* Attention, k est uniquement entier, contrairement à h.

| Jeu de données / Vote | k optimisé | top1 valid |
|:---------------------:|:----------:|:----------:|
| JDD 1 / Majoritaire  | 3 | 100% |
| JDD 1 / Unanime  | 2 | 99,8% |
| JDD 2 / Majoritaire  | 7 | 94.6% |
| JDD 2 / Unanime  | 2 | 87.2% |
| JDD 3 / Majoritaire  | 8 | 73.2% |
| JDD 3 / Unanime  | 2 | 51.6% |

In [9]:
# A compléter
# Fonction de cross validation
def cross_validation(N,data_x,data_y,data_labels,model_func,classes,vote_func,hmin=1,hmax=10,hstep=1):
    # Première étape : on divise les données en N datasets aléatoirement
    part = []
    for i in range(N):
        ids_app = [j for j in range(len(data_x)) if j%N != i]
        ids_val = [j for j in range(len(data_x)) if j%N == i]
        part.append({'app':[data_x[ids_app],data_y[ids_app],data_labels[ids_app]],
                     'val':[data_x[ids_val],data_y[ids_val],data_labels[ids_val]]})


    h_results = []
    for h in progressbar.progressbar(np.arange(hmin,hmax,hstep)):
        if h==0:
            continue
        h_top1 = []
        for i in range(N):
            model = model_func(classes,vote_func,h)
            app_x,app_y,app_labels = part[i]['app']
            val_x,val_y,val_labels = part[i]['val']
            model.learning(app_x,app_y,app_labels)
            top1,__,__ = model.test(val_x,val_y,val_labels,print_results=False)
            h_top1.append(top1)
        h_results.append((h,np.mean(h_top1)))
    return h_results

In [10]:
# A compléter
# Cross validation sur les 3 jeux de données, vote majoritaire et unanime
h_optimises = {}
for jeu in [1,2,3]:
    for vote in ['majoritaire','unanime']:
        # Chargement des données tp1 app et dec
        file_app = f'Archive/data_tp{jeu}_app.txt'
        file_dec = f'Archive/data_tp{jeu}_dec.txt'
        columns_labels = ['label','x','y']
        data_app = read_data(file_app,columns_labels)
        classes = np.unique(data_app['label'])

        # Apprentissage du classifieur avec les données d'apprentissage
        app_x = data_app['x']
        app_y = data_app['y']
        app_labels = data_app['label']

        h_results = cross_validation(3,app_x,app_y,app_labels,KPPV,classes,vote,2,10,1)
        h_max = max([p[1] for p in h_results])
        h_opti = [h for h in h_results if h[1]==h_max][0][0]
        top1_opti = [h for h in h_results if h[1]==h_max][0][1]
        h_optimises['{}_{}'.format(jeu,vote)] = (h_opti,top1_opti)
        

100% (8 of 8) |##########################| Elapsed Time: 0:00:04 Time:  0:00:04
100% (8 of 8) |##########################| Elapsed Time: 0:00:03 Time:  0:00:03
100% (8 of 8) |##########################| Elapsed Time: 0:00:03 Time:  0:00:03
100% (8 of 8) |##########################| Elapsed Time: 0:00:03 Time:  0:00:03
100% (8 of 8) |##########################| Elapsed Time: 0:00:03 Time:  0:00:03
100% (8 of 8) |##########################| Elapsed Time: 0:00:04 Time:  0:00:04


| Jeu de données / Vote | k optimisé | top1 valid |
|:---------------------:|:----------:|:----------:|
| JDD 1 / Majoritaire  | 3 | 100% |
| JDD 1 / Unanime  | 2 | 99,8% |
| JDD 2 / Majoritaire  | 7 | 94.6% |
| JDD 2 / Unanime  | 2 | 87.2% |
| JDD 3 / Majoritaire  | 8 | 73.2% |
| JDD 3 / Unanime  | 2 | 51.6% |

**2)** Quel k optimal obtenons-t-on pour un vote unanime ? Pourquoi ce résultat ? Et du coup, quel pourrait-être l'intérêt d'une cross validation pour un tel vote ?

**_Réponse :_** On obtient toujours le k minimum testé, à savoir 2 ici. C'est normal, si on augmente le k, on complexifie le problème, et une bonne détection avec un k élevé sera toujours bonne avec un k plus faible, et pas l'inverse automatiquement. Il y a cependant un intérêt à choisir un k élevé : si on préfère une non classification à une mauvaise prédiction, ce que la métrique top1 ne rend pas compte.

**3)** Evaluez maitenant vos classifieurs sur les jeux d'évaluation des 3 jeux de données, avec les 2 modes de vote, et avec les paramètres k optimisés. Reportez les résultats sur le tableau suivant. Les résultats sont-ils en raccord avec ceux obtenus sur les jeux de validations ? Comparez les performances des classifieurs en dissociant l'analyse sur les 3 jeux de données.

In [11]:
# A compléter
# Evaluation des classifieurs sur les jeux de tests
top1_eval = {}
for jeu in [1,2,3]:
    for vote in ['majoritaire','unanime']:
        h = h_optimises['{}_{}'.format(jeu,vote)][0]
        # Chargement des données tp1 app et dec
        file_app = f'Archive/data_tp{jeu}_app.txt'
        file_dec = f'Archive/data_tp{jeu}_dec.txt'
        columns_labels = ['label','x','y']
        data_app = read_data(file_app,columns_labels)
        data_dec = read_data(file_dec,columns_labels)
        classes = np.unique(data_app['label'])

        # Création du modèle
        model = KPPV(classes,vote,h)
        
        # Apprentissage du classifieur avec les données d'apprentissage
        app_x = data_app['x']
        app_y = data_app['y']
        app_labels = data_app['label']
        
        model.learning(app_x,app_y,app_labels)

        # Evaluation du classifieur avec les données d'évaluation
        dec_x = data_dec['x']
        dec_y = data_dec['y']
        dec_labels = data_dec['label']

        top1,top2,CM = model.test(dec_x,dec_y,dec_labels,print_results=False)
        top1_eval['{}_{}'.format(jeu,vote)] = top1

In [12]:
top1_eval

{'1_majoritaire': 0.996,
 '1_unanime': 0.99,
 '2_majoritaire': 0.946,
 '2_unanime': 0.884,
 '3_majoritaire': 0.696,
 '3_unanime': 0.506}

| Jeu de données / Vote | k | top1 valid | top1 eval |
|:---------------------:|:-:|:----------:|:---------:|
| JDD 1 / Majoritaire  | 3 | 100% | 99.6% |
| JDD 1 / Unanime  | 2 | 99,8% | 99% |
| JDD 2 / Majoritaire  | 7 | 94.6% | 94.6% |
| JDD 2 / Unanime  | 2 | 87.2% | 88.4% |
| JDD 3 / Majoritaire  | 8 | 73.2% | 69.6% |
| JDD 3 / Unanime  | 2 | 51.6% | 50.6% |

**_Réponse :_** Les résultats en évaluation sont sensiblement les mêmes que ceux en validation, donc les modèles n'ont pas de surapprentissage. Les résultats sont quasi parfaits sur le JDD 1, et très bons sur le JDD 2. Pour le JDD 3, c'est toujours compliqué...